In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.special import expit
import pickle
import csv
import copy
import itertools
from lightfm import LightFM
import lightfm.evaluation
import sys
sys.path.append('../')
import helpers
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse.linalg import spsolve
from sklearn.model_selection import train_test_split
import random
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import ndcg
import MAP
from functools import reduce
import matplotlib as plt 
from scipy.sparse import hstack
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score


/usr/local/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [16]:
# dataset nuevo 
path = './datasets_recsys/favorite_stories_books_sample.csv'
ff_data = pd.read_csv(path, sep=';', index_col=None, encoding='latin') 
ff_data['quantity'] = 1
ff_data = ff_data[['user_id','story_id','franchise', 'quantity']]

# abrimos dataset con features del usuario 
ff_users_features = pd.read_csv('./datasets_recsys/ff_user_features.csv')

# hacemos merge de ff_data con users_features 
ff_data = ff_data.merge(ff_users_features, on='user_id')

ff_data.head()

,user_id,story_id,franchise,quantity,q_written,q_favs,q_fav_authors,bio_words,communities
0,102390,7295117,Harry Potter,1,2,791,12,0,0
1,102390,10629488,Harry Potter,1,2,791,12,0,0
2,102390,10089646,Harry Potter,1,2,791,12,0,0
3,102390,8945573,Harry Potter,1,2,791,12,0,0
4,102390,8037808,Harry Potter,1,2,791,12,0,0


In [17]:
# cambiar path 
df_stories = pd.read_csv('/Users/andrescarvallo/Desktop/RecSys-Project-2017/datasets_recsys/ff_books_dataset.csv', sep='|', engine='python')

df_stories['story_id'] = pd.to_numeric(df_stories['story_id'], errors='coerce')
df_stories = df_stories.dropna(subset=['story_id'])

# convertimos a int la fila de story_id
id_story = pd.Series([int(float(x)) for x in list(df_stories['story_id'])])

# creamos nueva columna con ints story id
df_stories['story_id2'] = id_story.values

# proyectamos los datos que nos sirven en definitiva 
df_stories = df_stories[['story_id2', 'q_words', 'chapters', 'reviews', 'author']]

# cambiamos nombres de las columnas 
df_stories.columns = ['story_id', 'q_words', 'chapters', 'reviews', 'author']

df_stories.head()

,story_id,q_words,chapters,reviews,author
0,1895110,725.0,1.0,2.0,100016.0
1,1574594,1274.0,1.0,8.0,100016.0
2,767307,3100.0,3.0,39.0,100035.0
3,765982,3598.0,3.0,35.0,100035.0
4,396405,120.0,0.0,1.0,100062.0


In [30]:
df_stories.reviews.describe()

count    1.315218e+06
mean     3.933243e+01
std      2.177081e+02
min      0.000000e+00
25%      3.000000e+00
50%      7.000000e+00
75%      2.000000e+01
max      3.533600e+04
Name: reviews, dtype: float64

In [31]:
# creamos funciones para dar label a tramos de features 

def label_words(row):
    if 0< row['q_words'] < 1000 :
        return 0
    if 1000 <= row['q_words'] < 5000:
        return 1    
    if 5000 <= row['q_words'] < 12000:
        return 2
    else:
        return 3

def label_chapters(row):
    if 0 < row['chapters'] < 1:
        return 0
    if 1 < row['chapters'] < 2:
        return 1
    if 2 < row['chapters'] < 4:
        return 2
    else:
        return 3

def label_reviews(row):
    if 0 < row['reviews']< 10:
        return 0
    if 10 <= row['reviews'] < 20:
        return 1
    if 30 <= row['reviews'] < 50:
        return 2
    else:
        return 3

# en caso de error correr antes la proxima celda para que encuentre la columande pagerank
def label_pagerank(row):
    if 0 < row['page_rank'] < 0.00001:
        return 0
    if 0.00001 <= row['page_rank'] < 0.00003:
        return 1
    if 0.00003 <= row['page_rank'] < 0.0005:
        return 2
    else:
        return 3


### juntamos todos los datos en un df:

In [32]:
ff_final = ff_data.merge(df_stories, on='story_id')

# limpiamos caracteres en columna q_words
ff_final['q_words'] = pd.to_numeric(ff_final['q_words'], errors='coerce')
ff_final = ff_final.dropna(subset=['q_words'])

# limpiamos caracteres en columna chapters
ff_final['chapters'] = pd.to_numeric(ff_final['chapters'], errors='coerce')
ff_final = ff_final.dropna(subset=['chapters'])

# limpiamos caracteres en columna reviews
ff_final['reviews'] = pd.to_numeric(ff_final['reviews'], errors='coerce')
ff_final = ff_final.dropna(subset=['reviews'])

# convierte author id en int
id_author = pd.Series([int(float(x)) for x in list(ff_final['author'])])

# creamos nueva columna con ints author id
ff_final['author2'] = id_author.values


# dejamos columnas que nos interesan 
ff_final = ff_final[['user_id', 'story_id', 'quantity', 'q_words', 'chapters', 'reviews', 'author2', 'q_written','q_favs','q_fav_authors','bio_words','communities']]

# cambiamos nombre de columnas 
ff_final.columns = ['user_id', 'story_id', 'quantity', 'q_words', 'chapters', 'reviews', 'author','q_written','q_favs','q_fav_authors','bio_words','communities']

# PAGE RANK --> 
# abrimos df pagerank autor 
ff_pagerank = pd.read_csv('datasets_recsys/authors_pagerank.csv', sep=';')
ff_final = ff_final.merge(ff_pagerank, on='author')
#ff_final = ff_final[['user_id', 'story_id', 'quantity', 'q_words',  'chapters', 'reviews', 'author', 'page_rank']]

#creamos columnas de tramos para q_words, chapters, reviews y page rank 
ff_final['words_range'] = ff_final.apply (lambda row: label_words(row),axis=1)
ff_final['chapters_range'] = ff_final.apply (lambda row: label_chapters(row),axis=1)
ff_final['reviews_range'] = ff_final.apply (lambda row: label_reviews(row),axis=1)
#ff_final['pagerank_range'] = ff_final.apply (lambda row: label_pagerank(row),axis=1)

ff_final


,user_id,story_id,quantity,q_words,chapters,reviews,author,q_written,q_favs,q_fav_authors,bio_words,communities,page_rank,words_range,chapters_range,reviews_range
0,102390,7295117,1,45115.0,12.0,325.0,3140188,2,791,12,0,0,7.412535e-07,3,3,3
1,4767346,7295117,1,45115.0,12.0,325.0,3140188,0,2857,2,570,0,7.412535e-07,3,3,3
2,5695337,7295117,1,45115.0,12.0,325.0,3140188,1,241,24,19,0,7.412535e-07,3,3,3
3,5778823,7295117,1,45115.0,12.0,325.0,3140188,0,66,1,0,0,7.412535e-07,3,3,3
4,6861153,7295117,1,45115.0,12.0,325.0,3140188,2,224,7,0,0,7.412535e-07,3,3,3
5,8816804,7295117,1,45115.0,12.0,325.0,3140188,1,133,7,0,0,7.412535e-07,3,3,3
6,1568093,7295117,1,45115.0,12.0,325.0,3140188,0,237,10,0,0,7.412535e-07,3,3,3
7,1656852,7295117,1,45115.0,12.0,325.0,3140188,0,17,4,539,0,7.412535e-07,3,3,3
8,1912243,7295117,1,45115.0,12.0,325.0,3140188,0,624,1,0,0,7.412535e-07,3,3,3
9,1945346,7295117,1,45115.0,12.0,325.0,3140188,0,177,0,0,0,7.412535e-07,3,3,3


In [33]:
# creamos tramos 
def label_q_favs(row):
    if 5 < row['q_favs'] < 200:
        return 0
    if 200 <= row['q_favs'] < 600:
        return 1
    if 600 <= row['q_favs'] < 1200:
        return 2
    else:
        return 3

def label_q_fav_authors(row):
    if 0 < row['q_fav_authors'] < 20:
        return 0
    if 20 <= row['q_fav_authors'] < 40:
        return 1
    if 40 <= row['q_fav_authors'] < 90:
        return 2
    else:
        return 3

def label_bio_words(row):
    if 0 < row['bio_words'] < 250:
        return 0
    if 250 <= row['bio_words'] < 500:
        return 1
    if 500 <= row['bio_words'] < 750:
        return 2
    else:
        return 3

def label_q_written(row):
    if 0 < row['q_written'] < 1:
        return 0
    if 1 <= row['q_written'] < 2:
        return 1
    if 2 <= row['q_written'] < 20:
        return 2
    else:
        return 3   

In [34]:
# agregamos columnas adicionales para user features
ff_final['favs_range'] = ff_final.apply (lambda row: label_q_favs(row),axis=1)
ff_final['fav_authors_range'] = ff_final.apply (lambda row: label_q_fav_authors(row),axis=1)
ff_final['bio_words_range'] = ff_final.apply (lambda row: label_bio_words(row),axis=1)
ff_final['q_written_range'] = ff_final.apply (lambda row: label_q_written(row),axis=1)

In [35]:
# incluye usuarios e items que tengan al menos 5 interacciones
df = helpers.threshold_interactions_df(ff_final, 'user_id', 'story_id', 5, 5)

Interacciones iniciales user - fav_story
# filas: 15000
# columnas: 300171
Sparsity: 0.021%
Interacciones finales (luego de reduccion)
# filas: 10152
# columnas: 38863
Sparsity: 0.130%


In [36]:
# crea la matriz likes de user - item 
likes, uid_to_idx, idx_to_uid,mid_to_idx, idx_to_mid = helpers.df_to_matrix(df, 'user_id', 'story_id')

In [37]:
likes.shape

(10152, 38863)

In [38]:
# creamos matriz de train, testing y lista con id de usuarios
# ver funcion train_test_split en helpers.py 
train, test, user_index = helpers.train_test_split(likes, 5, fraction=0.2)

In [39]:
# hacemos una copia del training set para incluir usuarios con datos en el test set 
eval_train = train.copy()
non_eval_users = list(set(range(train.shape[0])) - set(user_index))

eval_train = eval_train.tolil()
for u in non_eval_users:
    eval_train[u, :] = 0.0
eval_train = eval_train.tocsr()
eval_train

<10152x38863 sparse matrix of type '<class 'numpy.float64'>'
	with 125784 stored elements in Compressed Sparse Row format>

### ahora queremos agregar mas features a la recomendacion:
Para eso ocupamos DictVectorizer (de scikitlearn) que toma una lista de diccionarios, donde cada diccionario contiene los nombres de los features como key y su weight como values. Aqui asumimos que cada weight es 1, y tomaremos combinacion de features == 1 cada vez que aparecen.
Por ejemplo si tiene min_reviews y max_words , o max_pagerank y min_words...

In [40]:
# creamos una lista de diccionarios que contengan features y 
#weights en el mismo orden que item_id (idx_to_mid) 
#(weight tiene valor 1 si aparece una vez, si no aparece no esta en el diccionario)

# features_stories: 
# words_range	chapters_range	reviews_range

feat_dlist = [{} for _ in idx_to_mid]

# recorre cada tupla de la df con features adicionales 
for idx, row in ff_final.iterrows():
    
    # features del usuario 
    # q_written, q_favs, q_fav_authors, bio_words, communities
    
    # consideramos feature adicional --> 'cantidad de reviews' , se pueden agregar fetures haciendo 
    # 'coocurrencia' i.e historia que tiene medium_reviews, max_words y min_chapters a esa le asigna 1
    # AQUI MODIFICAR FEATURES A CONSIDERAR
    feat_key = '{}_{}_{}'.format(row.words_range, row.chapters_range, row.reviews_range)
    idx = mid_to_idx.get(row.story_id)
    if idx is not None:
        feat_dlist[idx][feat_key] = 1

In [41]:
# diccionario que contiene un 1 si la historia tiene x reviews , esto es para construir una matriz
# item - feature
feat_dlist

[{'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_2.0': 1},
 {'2.0_3.0_1.0': 1},
 {'2.0_3.0_2.0': 1},
 {'1.0_3.0_3.0': 1},
 {'2.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'0.0_3.0_2.0': 1},
 {'2.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'0.0_3.0_2.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_2.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'0.0_3.0_2.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'2.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'3.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'1.0_3.0_2.0': 1},
 {'1.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'0.0_3.0_3.0': 1},
 {'2.0_3.0_3.0': 1},
 {'1.0_3.0_3.0': 1},
 {'2.0_3.0_3.

In [42]:
# metemos la lista de diccionarios en formato matriz con DictVectorizer
dv = DictVectorizer()
item_features = dv.fit_transform(feat_dlist)

Ahora tenemos una matrix de features para cada item, donde cada fila es un item (en el mismo orden que la matriz de likes...) y cada columna es un tag unico.

In [43]:
item_features.shape

(38863, 28)

Ahora para agregar los features tenemos que contruir la matriz de identidad y agregarsela al lado de la matriz que contiene los features de los items.

In [44]:
#hstack de los item_features
eye = sp.eye(item_features.shape[0], item_features.shape[0]).tocsr()
item_features_concat = sp.hstack((eye, item_features))
item_features_concat = item_features_concat.tocsr().astype(np.float32)

In [45]:
item_features_concat.shape

(38863, 38891)

### Recomendacion agregando nuevo features de historias:

**BPR (Bayesian Personalized Ranking Pairwise Loss):**  <br>
1. Escogemos un usuario random y una historia favorita, este es un item positivo (+). 
2. Escogemos un item j que no sea favorita del usuario random. Este es un item negativo. 
3. Utilizamos una ecuacion de factorizacion matricial para predecir el score de los items positivos (pui = score del usuario u del item i) , en el caso de factorizacion matricial será : xu⋅yi
4. Predecimos el score del usuario u y el item negativo j (puj).
5. Obtenemos las diferencias entre scores positivos y negativos --> xuij=pui−puj
6. Pasamos la diferencia a través de una funcion sigmoide (progresion temporal) y lo utilizamos para asignar un peso al update de los parametros del modelo via stochastic gradient descent (SGD) para que el modelo aprenda a rankear (learning to rank) 

**WARP(Weighted Approximate-Rank Pairwise Loss) --> es lo mismo que implicit feedback:**<br>
- WARP es similar a BPR: tomamos un sample de items positivos y negativos para un usuario random, predecimos para ambos y calculamos la diferencia. 
- En BPR hacemos un update de SGD (stochastis gradient descent) con la diferencia de positivos y negativos como un weight.  
- En WARP, solo corremos el update con SGD y predecimos si esta erroneo, es decir que el score del item negativo es mayor al del positivo.  
- Si la prediccion no fue incorrecta, continuamos iterando sobre items negativos hasta que prediccion este erronea o encontrar un valor de corte.

In [46]:
type(item_features_concat)
len(ff_final_user_id)

999

In [62]:
#import pickle

#ff_final['user_id'][0:1000].to_csv('ff_final_FM.csv', sep=',', index=False, header='user_id')
#pickle.dump(idx_to_mid, open('idx_to_mid_FM.pickle', 'wb'))
#sp.save_npz('./likes_FM.npz', likes)
#sp.save_npz('./item_features_concat_FM.npz', item_features_concat)
#sp.save_npz('./story_features_concat_FM.npz', item_features_concat)

In [10]:
ff_final_user_id = pd.read_csv('./ff_final_FM.csv', sep=',')
idx_to_mid = pickle.load(open('idx_to_mid_FM.pickle', 'rb'))
likes = sp.load_npz('./likes_FM.npz')

# aqui escoger dependiendo si queremos user features (item_features_concat_FM.npz) 
# o story features --> story_features_concat.npz
#item_features_concat = sp.load_npz('./item_features_concat_FM.npz')
item_features_concat = sp.load_npz('./story_features_concat_FM.npz')



In [13]:
import time
# recomienda al usuario index = 0,1,2,3,4, etc... de la lista de usuarios 

cont = 0 

data = []

# se puede cambiar a 'bpr' (bayesian personalised ranking) o 'warp'
model_features = LightFM(loss='bpr',
                         n=10, # maximo numero de sample de 'positives' para cada update.
                no_components = 300, # factores latentes 
                random_state=2016, # seed del pseudo random number generator 
                max_sampled = 10, # maximo numero de 'negative samples' (usuario NO interesado) 
                learning_rate=0.1, 
                        k=10,
                        item_alpha = 0, # castiga sobre features del user (si tiene)
                        user_alpha = 0) # castiga sobre features del item (si tiene)

# aqui agrega item_features_concat que incluye matriz con features adicionales
# item_features=item_features_concat dentro de fit()
%time model_features.fit(likes, epochs=1, num_threads=100)
t0 = time.time()

for user_id in list(ff_final_user_id['user_id']):

    user_index = cont
    n_items = len(list(idx_to_mid.values()))

  

    # predecimos para el user de index 10 del listado de todos los items
    scores_features = model_features.predict(user_index, np.arange(n_items))
    item_id = list(idx_to_mid.values())

    recommendations_features = list(zip(item_id, scores_features))

    # ordenamos y dejamos los top 5
    top5 = sorted(recommendations_features, key = lambda x: x[1])[-5:]

    items_recommended_features = []
    for x in top5:
        items_recommended_features.append(x[0])

    # lista reversa (para que nos quede el de mayor score primero)
    items_recommended_features = items_recommended_features[::-1]
    
    data.append([user_id,items_recommended_features])
    
    cont += 1
t1 = time.time()
    
print('Recomendacion total: {}\n Recomendacion por usuario: {}'.format((t1-t0), (t1-t0)/len(ff_final_user_id)))

df_results_features = pd.DataFrame(data, columns = ('user_id', 'rec_stories'))

df_results_features

# guardamos los resultados en un csv 

df_results_features.to_csv('FM_results_NO_features_PICKLE_10_iteraciones_300_factors.csv', index=False, sep=';')


    # ejemplo recomendacion top 5 score al usuario index 10 de la lista de users
    #print(items_recommended_features)

    # precision at k (para cada user que estan en train y test)
    # Measure the precision at k metric for a model: the fraction of known 
    # positives in the first k positions of the ranked list of results. A perfect score is 1.0.
    #precision_k = list(lightfm.evaluation.precision_at_k(model_features, 
                                               #test, train, 
                                               #k=5, 
                                               #item_features = item_features_concat))

    #precision_k_average = sum(precision_k)/len(precision_k)

    #print('precision@5 with features: {}'.format(precision_k_average))

CPU times: user 6.25 s, sys: 92.9 ms, total: 6.35 s
Wall time: 6.37 s
Recomendacion total: 111.5709228515625
 Recomendacion por usuario: 0.0371903076171875


In [12]:
df_result = pd.read_csv('FM_results_story_features_PICKLE_10_iteraciones.csv', sep=';')
df_result

,user_id,rec_stories
0,102390,"[9060508, 6560750, 11298223, 3157478, 10562882]"
1,4767346,"[3157478, 6560750, 2493456, 4536005, 2477819]"
2,5695337,"[9060508, 11298223, 6704012, 9681062, 10096096]"
3,5778823,"[4946293, 7965930, 5716145, 6550419, 5882420]"
4,6861153,"[9060508, 11298223, 6704012, 9681062, 10096096]"
5,8816804,"[6560750, 3157478, 9060508, 2477819, 9873249]"
6,1568093,"[4946293, 7965930, 5716145, 6550419, 7402590]"
7,1656852,"[4946293, 7965930, 9681062, 5716145, 5882420]"
8,1912243,"[2400483, 3157478, 10610076, 5276974, 9873249]"
9,1945346,"[4946293, 2125016, 9681062, 5276974, 6704012]"


### recomendacion SIN considerar features adicionales: 

In [ ]:
# recomienda al usuario index 0,1,2,3,4,etc... de la lista de usuarios 

cont = 0 

data = []

# se puede cambiar a 'bpr' (bayesian personalised ranking) o 'warp'
model = LightFM(loss='warp',
                         n=10, # maximo numero de sample de 'positives' para cada update.
                no_components = 100, # factores latentes 
                random_state=2016, # seed del pseudo random number generator 
                max_sampled = 10, # maximo numero de 'negative samples' (usuario NO interesado) 
                                  # durante 'warp' 
                learning_rate=0.01, 
                        k=10,
                        item_alpha = 0, # castiga sobre features del user (si tiene)
                        user_alpha = 0) # castiga sobre features del item (si tiene)

# aqui NO incluye item_features 
model.fit(likes, epochs=10,num_threads=8)


for user_id in ff_final['user_id'][0:10]:

    user_index = cont 
    
    n_items = len(list(idx_to_mid.values()))

    # predecimos para el user de index 10 del listado de todos los items
    scores = model.predict(user_index, np.arange(n_items))
    item_id = list(idx_to_mid.values())

    recommendations = list(zip(item_id, scores))

    # ordenamos y dejamos los top 5 
    top5 = sorted(recommendations, key = lambda x: x[1])[-5:]

    items_recommended = []
    for x in top5:
        items_recommended.append(x[0])

    # lista reversa (para que nos quede el de mayor score primero)
    items_recommended = items_recommended[::-1]
    
    data.append([user_id,items_recommended_features])
    
    cont += 1

df_results_no_features = pd.DataFrame(data, columns = ('user_id', 'rec_stories'))

df_results_no_features.to_csv('resultados_fm_onerun.csv', sep=',', index=False)

    # recomienda items top 10 score para user con index 10 
    #print(items_recommended)

    #precision_k = list(lightfm.evaluation.precision_at_k(model, 
     #                                          test, train, 
      #                                         k=5))

    #precision_k_average = sum(precision_k)/len(precision_k)

    #print('precision@5 modelo inicial (sin features): {}'.format(precision_k_average))

### vemos que precision@10 sin features nos da mas bajo
Con pagerank + author mejora más aun la recomendacion , pero pagerank solo no aporta 

ver como mejora al incoporar features por separado...

**features**       | **precision@5** <br>
no features        | 0.020 <br>
pagerank + author  | 0.02916 <br>
author             | 0.02916 <br>
pagerank           | 0.0309 <br> 
q_reviews          | 0.02256 <br>
q_words            | 0.02768 <br>
q_chapters         | 0.015 <br>
reviews+words+chap | 0.0299 <br> 

## ocupando bins (i.e entre 0-10 low, 10-50 medium...) dismuye precision

In [ ]:
# probar aleatorio con la misma libreria light fm 
# calcular ndcg, esta precision de lightfm esta calculada como AUC (area bajo la curva)

# incluir features del usuario (sentimiento de la biografia), diversidad de historias favoritas,
# q_favoritas, q_franquicias distintas a las que sigue, diversidad de franquicias...  